<a href="https://colab.research.google.com/github/PerdikosFilippos/EEG-Classifications-and-Models/blob/main/EEG_Epilapsy_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne==1.0.3


#Download data
* Read the paper to understand data.
* Datasets were collected using EMOTIVE with the 128 Hz sampling frequency.
* There are two differnt dataset collected in Guinea Bissau (97 subjects) and Nigeria (112 subjects).
* Here we are dealing with data collected in Guinea Bissau


In [2]:
!wget https://zenodo.org/record/1252141/files/EEGs_Guinea-Bissau.zip


--2024-02-06 09:36:38--  https://zenodo.org/record/1252141/files/EEGs_Guinea-Bissau.zip
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1252141/files/EEGs_Guinea-Bissau.zip [following]
--2024-02-06 09:36:39--  https://zenodo.org/records/1252141/files/EEGs_Guinea-Bissau.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 153973086 (147M) [application/octet-stream]
Saving to: ‘EEGs_Guinea-Bissau.zip.1’

EEGs_Guinea-Bissau. 100%[===================>] 146.84M  22.7MB/s    in 7.7s    

2024-02-06 09:36:47 (19.2 MB/s) - ‘EEGs_Guinea-Bissau.zip.1’ saved [153973086/153973086]



#Unzip Data

In [3]:
#unzip the files
from zipfile import ZipFile
data = ZipFile('EEGs_Guinea-Bissau.zip')
data.extractall()

#Read Data

In [4]:


import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [5]:

meta_df=pd.read_csv('https://zenodo.org/record/1252141/files/metadata_guineabissau.csv')
meta_df.head()

,subject.id,Group,Eyes.condition,Remarks,recordedPeriod,startTime
0,1,Epilepsy,closed-3min-then-open-2min,by 45s reposition electrodes,301,27/5/2020 14:33
1,2,Control,open-3min-then-closed-2min,NaN,309,26/5/2020 22:44
2,3,Epilepsy,closed-3min-then-open-2min,NaN,309,27/5/2020 14:26
3,4,Epilepsy,closed-3min-then-open-2min,"Green lights not shown, but good EEG traces",299,27/5/2020 15:23
4,5,Control,closed-3min-then-open-2min,NaN,302,23/5/2020 19:09


In [6]:
#now i need to seprate Epilepsy vs Control subjects
EP_sub=meta_df['subject.id'][meta_df['Group']=='Epilepsy']
CT_sub=meta_df['subject.id'][meta_df['Group']=='Control']

In [7]:

#read csv files
Epilepsy=[pd.read_csv('EEGs_Guinea-Bissau/signal-{}.csv.gz'.format(i), compression='gzip') for i in  EP_sub]
Control=[pd.read_csv('EEGs_Guinea-Bissau/signal-{}.csv.gz'.format(i), compression='gzip') for i in  CT_sub]

In [8]:
Epilepsy[0].head()


,Unnamed: 0,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,...,CQ_F3,CQ_P7,CQ_P8,CQ_F4,CQ_AF3,CQ_FC5,CQ_O1,CQ_T8,CQ_F8,CQ_DRL
0,1,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,...,4,4,4,4,4,4,4,4,4,4
1,2,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,...,4,4,4,4,4,4,4,4,4,4
2,3,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,...,4,4,4,4,4,4,4,4,4,4
3,4,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,...,4,4,4,4,4,4,4,4,4,4
4,5,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,...,4,4,4,4,4,4,4,4,4,4


In [9]:
#remove non eeg channels
Epilepsy=[i.iloc[:,1:15] for i in  Epilepsy]
Control=[i.iloc[:,1:15] for i in  Control]

#Convert to MNE Object

In [10]:
import mne
def convertDF2MNE(sub):
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128)
    info.set_montage('standard_1020')
    data=mne.io.RawArray(sub.T, info)
    data.set_eeg_reference()
    data.filter(l_freq=0.1,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    epochs=epochs.drop_bad()

    return epochs

In [11]:
%%capture
#Convert each dataframe to mne object
Epilepsy=[convertDF2MNE(i) for i in  Epilepsy]
Control=[convertDF2MNE(i) for i in  Control]

In [12]:
%%capture
#concatenate the epochs
Epilepsy_epochs=mne.concatenate_epochs(Epilepsy)
Control_epochs=mne.concatenate_epochs(Control)

#Create Labels and Groups

In [13]:
Epilepsy_group=np.concatenate([[i]*len(Epilepsy[i]) for i in range(len(Epilepsy))])#create a list of list where each sub list corresponds to subject_no
Control_group=np.concatenate([[i]*len(Control[i]) for i in range(len(Control))])#create a list of list where each sub list corresponds to subject_no

Epilepsy_label=np.concatenate([[0]*len(Epilepsy[i]) for i in range(len(Epilepsy))])
Control_label=np.concatenate([[1]*len(Control[i]) for i in range(len(Control))])

In [14]:
Epilepsy_group.shape,Control_group.shape,Epilepsy_label.shape,Control_label.shape


((3995,), (3461,), (3995,), (3461,))

In [15]:
#combine data
data=mne.concatenate_epochs([Epilepsy_epochs,Control_epochs])
group=np.concatenate((Epilepsy_group,Control_group))
label=np.concatenate((Epilepsy_label,Control_label))
print(len(data),len(group),len(label))

Not setting metadata
7456 matching events found
No baseline correction applied
0 bad epochs dropped
7456 7456 7456


#Feature Extraction - Power spectral density
The power spectral density of a signal is a measure of how much power the signal has at each different frequency. The power spectral density (power spectrum) reflects the ‘frequency content’ of the signal or the distribution of signal power over frequency.

In [16]:
# source: https://mne.tools/stable/auto_tutorials/clinical/60_sleep.html#sphx-glr-auto-tutorials-clinical-60-sleep-py
from mne.time_frequency import psd_welch
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data.
    """
    # specific frequency bands
    FREQ_BANDS = {"delta": [0.5, 4.5],
                  "theta": [4.5, 8.5],
                  "alpha": [8.5, 11.5],
                  "sigma": [11.5, 15.5],
                  "beta": [15.5, 30],
                  "gamma": [30, 45],
                  }

    psds, freqs = psd_welch(epochs, picks='eeg', fmin=0.5, fmax=45)# Compute the PSD using the Welch method
    psds /= np.sum(psds, axis=-1, keepdims=True)    # Normalize the PSDs

    X = []#For each frequency band, compute the mean PSD in that band
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)# Compute the mean PSD in each frequency band.
        X.append(psds_band)

    return np.concatenate(X, axis=1)#Concatenate the mean PSDs for each band into a single feature vector

#Classification 5-fold


In [21]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [24]:
%%capture
features=[]
for d in range(len(data)):#get features from each epoch and save in a list
  features.append(eeg_power_band(data[d]))

In [25]:
#convert list to array
features=np.concatenate(features)
features.shape

(7456, 84)

In [26]:
#do 5 fold cross validation
clf=RandomForestClassifier()
accuracies=cross_val_score(clf, features,label,groups=group,cv=5)
print('Five fold accuracies',accuracies)
print('Average accuracy',np.mean(accuracies))

Five fold accuracies [0.70576408 0.68879946 0.657277   0.72501677 0.6861167 ]
Average accuracy 0.692594800258207
